In [103]:
%%capture
import pandas as pd
import numpy as np

import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaMulticore
from gensim.models.phrases import Phraser, Phrases

import spacy
spacy.load('en', disable=['parser', 'ner'])

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("english")
stop_words.extend(['say', 'would', 'also', 'use'])

from pprint import pprint

from google.colab import drive

In [104]:
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [108]:
# Load data
data = pd.read_csv(r'/content/gdrive/My Drive/Datasets/Text - News Dataset/articles1.csv')

In [109]:
data.head(3)

,Unnamed: 0,Unnamed: 0.1,id,title,publication,author,date,year,month,url,content
0,23613,23617,42368,"Zuckerberg Gets Hacked, Turns Out to Have Really Lame Password - Breitbart",Breitbart,John Hayward,2016-06-06,2016.0,6.0,NaN,"Over the weekend, Twitter and Pinterest accounts belonging to Facebook founder Mark Zuckerberg were hacked by a group called the “OurMine Team,” reportedly based in Saudi Arabi..."
1,39498,40983,59854,"Turkish bombing: 13 soldiers killed, 55 people wounded",CNN,Madison Park,2016-12-17,2016.0,12.0,NaN,"(CNN) A car bomb exploded near a public bus in Turkey on Saturday, killing 13 soldiers and wounding 55 people, according to Turkish officials. Authorities later arrested 15 p..."
2,13203,13204,31940,"Fmr UFC Fighter Tim Kennedy Reenlists, Credits Trump: The Military ’Got Our Teeth Back’ - Breitbart",Breitbart,Trent Baker,2017-04-17,2017.0,4.0,NaN,"Sunday on “Fox Friends Weekend,” former UFC Fighter Tim Kennedy said he decided to in the U. S. Army Special Forces after noticing the change in leadership under new Preside..."


In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    4000 non-null   int64  
 1   Unnamed: 0.1  4000 non-null   int64  
 2   id            4000 non-null   int64  
 3   title         4000 non-null   object 
 4   publication   4000 non-null   object 
 5   author        3515 non-null   object 
 6   date          4000 non-null   object 
 7   year          4000 non-null   float64
 8   month         4000 non-null   float64
 9   url           0 non-null      float64
 10  content       4000 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 343.9+ KB


In [111]:
def preprocess_text(texts, allowed_pos_tags=['NOUN', 'VERB', 'ADV', 'ADJ']):
  preprocessed_texts = list()
  # Use simple preprocess to convert to lower case and remove punctuation then add to list if NOT in list of stop words
  texts = [[token for token in simple_preprocess(doc, deacc=True) if token not in stop_words] for doc in texts]
  bigrams = Phrases(texts, min_count=2)
  trigrams = Phrases(bigrams[texts], min_count=2)
  bigram_model = Phraser(bigrams)
  trigram_model = Phraser(trigrams)
  bi_gram_texts = [bigram_model[doc] for doc in texts]
  tri_gram_texts = [trigram_model[doc] for doc in bi_gram_texts]

  nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'] )
  for doc in tri_gram_texts:
    doc = nlp(' '.join(doc))
    # Use spacy to lemmatize and conform token's pos tag is in the allowed list
    preprocessed_texts.append([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
  return preprocessed_texts

In [112]:
# Cleaned up text
texts = preprocess_text(data.content)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [113]:
# Create dictionary
id2word = corpora.Dictionary(texts)

# Create corpus
corpus = [id2word.doc2bow(text) for text in texts]

**Build LDA Model**

In [114]:
%%capture
# Get the coherence score for different number of topics
history = dict()
for i in range(20, 31):
  # Build LDA model with i number of topcics
  lda_model = LdaMulticore(corpus=corpus, num_topics=i, id2word=id2word, random_state=1, passes=5)
  # Get topic coherence and add to history
  coherence_model = gensim.models.CoherenceModel(model=lda_model, texts=texts, corpus=corpus, dictionary=id2word)
  history[i] = coherence_model.get_coherence()

In [115]:
sorted(history.items(), key=lambda x: x[1], reverse=True)

[(20, 0.2830320464778939),
 (21, 0.27775344755493225),
 (23, 0.2764905216725714),
 (29, 0.27553904672756724),
 (22, 0.27291396023113595),
 (25, 0.2715243535102446),
 (28, 0.2661245151509894),
 (24, 0.26556283567460676),
 (30, 0.26510541240680496),
 (26, 0.264933749826053),
 (27, 0.26186433924385494)]

In [116]:
%%capture
# Build LDA model with number of topics that had the highest coherence score
lda_model = LdaMulticore(corpus=corpus, num_topics=20, id2word=id2word, random_state=1, passes=5)

In [117]:
pprint(lda_model.print_topics())

[(0,
  '0.015*"say" + 0.009*"company" + 0.007*"go" + 0.006*"make" + 0.005*"get" + '
  '0.005*"time" + 0.005*"people" + 0.005*"could" + 0.005*"work" + '
  '0.005*"year"'),
 (1,
  '0.011*"say" + 0.006*"report" + 0.006*"woman" + 0.005*"vote" + '
  '0.004*"company" + 0.004*"come" + 0.003*"sale" + 0.003*"call" + 0.003*"make" '
  '+ 0.003*"sell"'),
 (2,
  '0.011*"film" + 0.008*"show" + 0.007*"say" + 0.005*"make" + 0.004*"movie" + '
  '0.004*"play" + 0.004*"may" + 0.004*"star" + 0.004*"new" + 0.004*"report"'),
 (3,
  '0.009*"say" + 0.006*"customer" + 0.005*"people" + 0.004*"state" + '
  '0.004*"insurer" + 0.004*"make" + 0.004*"coverage" + 0.003*"pay" + '
  '0.003*"season" + 0.003*"drug"'),
 (4,
  '0.025*"say" + 0.008*"go" + 0.007*"people" + 0.006*"think" + 0.006*"know" + '
  '0.006*"make" + 0.004*"get" + 0.004*"police" + 0.004*"work" + 0.004*"take"'),
 (5,
  '0.009*"say" + 0.005*"go" + 0.005*"may" + 0.004*"state" + 0.004*"make" + '
  '0.004*"work" + 0.004*"people" + 0.004*"even" + 0.004*"trum

In [118]:
# Visualize topics
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.090357  0.011714       1        1  18.205437
4     -0.055006 -0.021608       2        1  12.537697
12    -0.056919  0.007982       3        1  11.507908
0     -0.045834  0.075315       4        1   9.224033
6     -0.056025 -0.036702       5        1   7.526011
9     -0.041443  0.021232       6        1   5.982751
8     -0.057690  0.021222       7        1   5.950605
18    -0.043504 -0.080245       8        1   5.078794
13    -0.055635 -0.035481       9        1   4.113034
15    -0.030190  0.039262      10        1   3.894242
5     -0.028757  0.060405      11        1   3.715602
14    -0.031893 -0.046327      12        1   3.326783
2     -0.008080  0.040076      13        1   2.228125
7      0.024190 -0.148075      14        1   1.923007
3      0.064079  0.074981      15        1   1.080528
19     0.086284  0.040007      16        1   0.949856
1      0.057711  0.006796      17        1   0.939268
16     0.077811  0.005190      18        1   0.740390
10     0.105572 -0.016159      19        1   0.663443
11     0.185684 -0.019586      20        1   0.412485, topic_info=                                         Term          Freq  ...  logprob  loglift
201                                     trump   5392.000000  ...  30.0000  30.0000
83                                        say  16546.000000  ...  29.0000  29.0000
9                                      attack   1433.000000  ...  28.0000  28.0000
53                                       make   3511.000000  ...  27.0000  27.0000
65                                     people   4287.000000  ...  26.0000  26.0000
154                                     state   1996.000000  ...  25.0000  25.0000
147                                    report   1771.000000  ...  24.0000  24.0000
55                                        may   2590.000000  ...  23.0000  23.0000
749                                   company   1536.000000  ...  22.0000  22.0000
294                                     woman   1392.000000  ...  21.0000  21.0000
900                                       win    730.000000  ...  20.0000  20.0000
89                                       show   1757.000000  ...  19.0000  19.0000
1                                      accord   1530.000000  ...  18.0000  18.0000
186                                      isis    501.000000  ...  17.0000  17.0000
187                                      kill    745.000000  ...  16.0000  16.0000
790                                      film    380.000000  ...  15.0000  15.0000
125                                   country   2005.000000  ...  14.0000  14.0000
332                                     first   1698.000000  ...  13.0000  13.0000
2234                                     vote   1186.000000  ...  12.0000  12.0000
191                                       new   1808.000000  ...  11.0000  11.0000
96                                       team    863.000000  ...  10.0000  10.0000
337                                        go   3829.000000  ...   9.0000   9.0000
367                                      play    823.000000  ...   8.0000   8.0000
1058                                     game    519.000000  ...   7.0000   7.0000
393                                      take   2601.000000  ...   6.0000   6.0000
100                                      time   2680.000000  ...   5.0000   5.0000
124                                      come   2529.000000  ...   4.0000   4.0000
463                                      many   1971.000000  ...   3.0000   3.0000
43                                    include   2191.000000  ...   2.0000   2.0000
111                                      year   1939.000000  ...   1.0000   1.0000
379                                    romney    135.611001  ...  -6.9612   1.6701
3495                              clinton_say     40.910066  ...  -8.1596   1.6644
14066                         

In [120]:
# Iterate through corpus to get dominant topic in each document
dominate_topics = list()
for i, row in enumerate(lda_model[corpus]):
  # Select the topic number and percent contribution of the dominant topic
  row = sorted(row, key=lambda x: x[1], reverse=True)[0]
  # Use index to append the text of the document along with the topic number and percent contribution
  dominate_topics.append([' '.join(texts[i]), row[0], row[1]])

# Create a dataframe using with the list from above
topic_sentences_df = pd.DataFrame()
topic_sentences_df[['text', 'topic number', 'percent contribution']] = np.array(dominate_topics, dtype=object)

In [121]:
pd.options.display.max_rows = None
# Group by topic number and show the three documents with the highest contribution for each topic number
topic_sentences_df.groupby('topic number').apply(lambda df: df.sort_values('percent contribution', ascending=False)[:3].drop(columns='topic number', axis=1))

text  percent contribution
topic number                                                                                                                                                                                                                
0            2130  many american_citie transform millennial generation downtown turn bustling neighborhood full new apartment pricey coffee bar soon city may start run millennial number demograph...              0.997907
             2675  last_fall apple chief design officer ask think draw make stuff friend have charlie_rose maybe ornament know installation design have have say responsible design apple include l...              0.997894
             1794  big carmaker close huge global player come constantly build expensive year existence want tesla lately take exhort company remain lean mean painting little_guy constant_threat ...              0.997754
1            3264  travel remote colony eastern lowland southernmost reach vast offer agricultural giant soybean mennonite resident descendant european peasant carve settlement thick forest year ...              0.998907
             3788  expert witness testify house_panel loot destruction priceless antique jihadist_groups across_middle_east identify collector europe_united_state top buyer ancient_artifact illeg...              0.997432
             3003  dramatic scientist tick forward new set midnight indicate probability global catastrophe high say project physicist universally recognize indicator world vulnerability catastro...              0.996215
2            2679  film base jauntily cynical play may startle moment wisecrack silence patter raise mayor thinking write film humor frequently harsh probably version paper likely review hack kee...              0.998747
             3173  officially say former male identity introduce quite literally way major hollywood figure could via cover preview connection mine muse sadly probably way ever agree put woman si...              0.995153
             1439  freewheel comic career advertising garner worldwide acclaim satirical entertainment abound tv radio record die freberg die hospital confirm always hero carry brilliant legacy f...              0.994780
3            2609  tourist stroll yellowstone active geyser take selfie front thousand bubble boil mud pot hiss steam vent tread planet great time bomb enormous puzzle geophysicist decade use sei...              0.997376
             2882  strength become chain mobile_ordering allows_customer order pay smartphone enter store cause customer take complain long point mobile_ordering starbuck drink ready time wait li...              0.997309
             2136  avoid insuring mortgage practice know redlining exacerbate city redline long outlaw hurt black homeowner accord federal_lawsuit_filed nonprofit represent time government fuel r...              0.997068
4            3278  love couple add twist formula contemplate endure rattlepate true never_met person kinkier reach parallel cultural afterlive illuminate process great author transform icon belov...              0.998069
             2221  come often enough seem variation painful pattern carnage follow disbelief condemnation condolence shrine flower candle letter dead yet see attack element may buoy terrorist epi...              0.997935
             2235  live years_old make daily_task take sip coffee task talk much embarrassment talk clear shaking lot say thank hold confidence take frustration life see change lebenthal day go l...              0.997912
5            2095  drop average pound report participant burn fewer_calorie day people age size regain percent lost_weight show finale diet industry react defensively argue participant lost_weigh...              0.998814
             2019  many_people_believe thing demonstrably false question propagate falsehood crime_statistic population buy collective delusion new sole province political right plenty liberal be...              0.99